In [1]:
# Run this code block just once after the kernel starts
# to change current directory to root

import sys
import os

sys.path.append("..")
os.chdir("..")
print(os.getcwd())

/home/student/bev-project


In [2]:
from criterion import OccupancyCriterion, prior_uncertainty_loss
from nuscenes_utilities import NUSCENES_CLASS_NAMES
from utilities import torch as torch_utils
from configs.config_utilities import load_config

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from dataset import NuScenesDataset
import numpy as np

In [3]:
device = torch_utils.detect_device()
print(device)

cuda


In [4]:
config = load_config("configs/configs.yml")

In [5]:
torch.cuda.is_available()

True

In [6]:
logits = torch.randn((1,2,3,3))
print(logits)

tensor([[[[ 1.2336,  0.5960,  0.8254],
          [ 1.2346,  0.3112, -1.0722],
          [ 0.3542,  1.3964, -0.8993]],

         [[-0.3451,  0.8739, -0.2343],
          [ 0.9797, -1.3031,  0.2852],
          [ 0.5566,  0.9894, -1.9713]]]])


In [7]:
labels = torch.tensor(
    [[[[ 0, 0, 0 ],
       [ 0, 1, 0 ],
       [ 0, 0, 0 ]],

      [[ 0, 0, 0 ],
       [ 0, 0, 0 ],
       [ 1, 1, 1 ]]]])

masks = torch.tensor(
    [[[ 0, 1, 0 ],
      [ 0, 1, 0 ],
      [ 0, 1, 0 ]]])

print("labels", labels.shape)
print("masks", masks.shape)

labels torch.Size([1, 2, 3, 3])
masks torch.Size([1, 3, 3])


In [8]:
criterion = OccupancyCriterion(
    config.prior,
    config.xent_weight,
    config.uncert_weight,
    config.weight_mode,
).to(device)

In [9]:
print(type(config.prior))
print(config.prior)

<class 'list'>
[0.44679, 0.02407, 0.14491, 0.02994, 0.02086, 0.00477, 0.00156, 0.00189, 0.00084, 0.00119, 0.00019, 0.00012, 0.00031, 0.00176]


In [10]:
priors = torch.tensor([0.5, 0.1])
class_weights = torch.sqrt(1 / priors)

In [11]:
uncert_loss = prior_uncertainty_loss(logits, masks, priors)
print(uncert_loss)

tensor(-1.0819)


/home/student/miniconda3/envs/senior_project/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [12]:
print(logits.new(priors))
print(logits.new(priors).view(1,-1,1,1))
print(logits.new(priors).view(1,-1,1,1).expand_as(logits))

tensor([0.5000, 0.1000])
tensor([[[[0.5000]],

         [[0.1000]]]])
tensor([[[[0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000]],

         [[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]]]])


In [13]:
priors_new = logits.new(priors).view(1, -1, 1, 1).expand_as(logits)
print(priors_new.shape)

torch.Size([1, 2, 3, 3])


In [18]:
xent = F.binary_cross_entropy_with_logits(logits, priors_new, reduce=False)
print(xent)

tensor([[[[0.7004, 0.7094, 0.7135],
          [1.3011, 0.7127, 0.7483],
          [0.6932, 1.2940, 1.0427]],

         [[0.4390, 1.2511, 1.2207],
          [0.3256, 0.3968, 0.5391],
          [0.3291, 0.6454, 1.4162]]]])


/home/student/miniconda3/envs/senior_project/lib/python3.8/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


In [24]:
masks

tensor([[[0, 1, 0],
         [0, 1, 0],
         [0, 1, 0]]])

In [23]:
masks.logical_not()

tensor([[[ True, False,  True],
         [ True, False,  True],
         [ True, False,  True]]])

In [29]:
masks.logical_not().float()

tensor([[[1., 0., 1.],
         [1., 0., 1.],
         [1., 0., 1.]]])

In [28]:
masks.logical_not().float()

torch.Size([1, 1, 3, 3])

In [30]:
(xent * masks.logical_not().float().unsqueeze(1)).mean()

tensor(0.5260)

In [31]:
~masks

tensor([[[-1, -2, -1],
         [-1, -2, -1],
         [-1, -2, -1]]])

In [16]:
dataset = NuScenesDataset(
    nuscenes_dir=config.nuscenes_dir,
    nuscenes_version=config.nuscenes_version,
    label_dir=config.label_dir,
    # sample_tokens=config.val_tokens,
    sample_tokens=np.loadtxt("configs/mini_val_sample_tokens.csv", dtype=str),
    image_size=(200, 112),
)
dataset_loader = DataLoader(
    dataset,
    batch_size=config.batch_size,
    num_workers=config.num_workers,
    pin_memory=True,
    shuffle=True,
)

--------------------------------------------------
Loading NuScenes version v1.0-mini ...
--------------------------------------------------


In [17]:
image, label, mask, calib = next(iter(dataset_loader))

In [19]:
mask.shape

torch.Size([10, 196, 200])